[Jeder kann coden](../../abstract/Contents.de.ipynb) / [Programmieren & TicTacToe](../../Programming_And_TicTacToe.de.ipynb) / [C# Einführung](../CSharp_Introduction.de.ipynb)

# Das Problem der speisenden Philosophen

<table border="0">
  <tr>
    <td>
        <img src="Dining_Philosophers.jpeg">
    </td>
    <td rowspan="2">
        <a href="https://miro.com/app/board/uXjVPm0lbTo=/?moveToWidget=3458764607889708296&cot=14"><img src="Radar_Dining_Philosophers.jpg"></a>
    </td>
  </tr>
  <tr>
    <td>
      <a href="https://de.wikipedia.org/wiki/Philosophenproblem">Das Problem der speisenden Philosophen – Wikipedia</a><br>
      <a href="https://learn.microsoft.com/de-de/dotnet/standard/threading/">Multithreading in .NET – Microsoft Learn</a><br>
      <a href="https://www.baeldung.com/cs/dining-philosophers">Dining Philosophers Problem Explained – Baeldung</a><br>
      <a href="https://docs.microsoft.com/de-de/dotnet/api/system.threading.semaphoreslim">SemaphoreSlim in .NET – Microsoft Docs</a><br>
      <a href="https://www.geeksforgeeks.org/deadlock-in-operating-system/">Deadlock in Betriebssystemen – GeeksForGeeks</a><br>
      <a href="https://www.javatpoint.com/deadlock-in-operating-system">Was ist ein Deadlock? – JavaTpoint</a><br>
      <a href="https://www.codeproject.com/Articles/10546/Dining-Philosophers-Solution-using-C">C# Beispiel: Dining Philosophers – CodeProject</a><br>
      <a href="https://refactoring.guru/de/design-patterns/concurrency">Design Patterns für Concurrency – Refactoring.Guru</a><br>
      <a href="https://dotnet.microsoft.com/en-us/learn/dotnet/what-is-dotnet">Was ist .NET? – Microsoft</a><br>
    </td>
  </tr>
</table>

Das **Problem der speisenden Philosophen (Dining Philosophers Problem)** und das Konzept des **Deadlocks** sind klassische Themen der nebenläufigen Programmierung (Concurrency). Sie illustrieren Herausforderungen bei der **Synchronisation** von Prozessen/Threads, die gemeinsame Ressourcen nutzen.

<a href="https://miro.com/app/board/o9J_lOJi2o0=/?moveToWidget=3074457361077289431&cot=14"><img src="Dining philosophers & deadlock.jpg"></a>

### Szenario:

* Es sitzen **fünf Philosophen** an einem runden Tisch.
* Zwischen jedem Paar von Philosophen liegt **eine Gabel** (insgesamt 5 Gabeln).
* Jeder Philosoph benötigt **zwei Gabeln** (die linke und rechte), um zu essen.
* Philosophen denken, essen und wiederholen diesen Zyklus.
* Eine Gabel kann **immer nur von einem Philosophen gleichzeitig benutzt** werden.

### Ziel:

Vermeide Situationen, in denen Philosophen **verhungern** oder sich **gegenseitig blockieren** (Deadlock).

## Was ist ein Deadlock?

Ein **Deadlock (Verklemmung)** tritt auf, wenn mehrere Threads auf Ressourcen warten, die gegenseitig blockiert sind, sodass **keiner mehr weiterarbeiten kann**.

### Vier Bedingungen, die zu einem Deadlock führen:

1. **Wechselseitiger Ausschluss** – Ressourcen können nur exklusiv genutzt werden.
2. **Hold and Wait** – Prozesse halten Ressourcen und warten auf weitere.
3. **Keine Präemption** – Ressourcen können nicht erzwungen weggenommen werden.
4. **Zirkuläres Warten** – Eine zyklische Kette von Wartenden entsteht.

## Beispiel für einen Deadlock (C#-Pseudocode)

```csharp
object gabel1 = new object();
object gabel2 = new object();

void PhilosophA()
{
    lock (gabel1)
    {
        Thread.Sleep(100); // Kurze Pause
        lock (gabel2)
        {
            Console.WriteLine("Philosoph A isst.");
        }
    }
}

void PhilosophB()
{
    lock (gabel2)
    {
        Thread.Sleep(100); // Kurze Pause
        lock (gabel1)
        {
            Console.WriteLine("Philosoph B isst.");
        }
    }
}
```

### Problem:

Wenn **Philosoph A `gabel1`** hält und **Philosoph B `gabel2`**, und beide die jeweils andere Gabel wollen, **entsteht ein Deadlock**.

## Wie kann man Deadlocks vermeiden?

### Ressourcenreihenfolge einhalten (Kanonische Ordnung)

Ordne Ressourcen global und lasse alle Threads diese **immer in derselben Reihenfolge** anfordern:

```csharp
object gabel1 = new object();
object gabel2 = new object();

void PhilosophA()
{
    lock (gabel1)
    {
        lock (gabel2)
        {
            Console.WriteLine("Philosoph A isst.");
        }
    }
}

void PhilosophB()
{
    lock (gabel1) // Statt gabel2 zuerst
    {
        lock (gabel2)
        {
            Console.WriteLine("Philosoph B isst.");
        }
    }
}
```

### TryLock mit Timeout verwenden

Nutze `Monitor.TryEnter` mit Timeout, um **nicht zu blockieren**, wenn eine Ressource nicht verfügbar ist:

```csharp
if (Monitor.TryEnter(gabel1, TimeSpan.FromMilliseconds(100)))
{
    try
    {
        if (Monitor.TryEnter(gabel2, TimeSpan.FromMilliseconds(100)))
        {
            try
            {
                Console.WriteLine("Philosoph isst.");
            }
            finally { Monitor.Exit(gabel2); }
        }
    }
    finally { Monitor.Exit(gabel1); }
}
else
{
    Console.WriteLine("Philosoph wartet weiter.");
}
```

## Fazit

| Begriff                 | Bedeutung                                                                                                    |
| ----------------------- | ------------------------------------------------------------------------------------------------------------ |
| **Dining Philosophers** | Modelliert Ressourcenkonflikte bei mehreren konkurrierenden Prozessen.                                       |
| **Deadlock**            | Verklemmungssituation, in der mehrere Prozesse sich gegenseitig blockieren.                                  |
| **Lösungsansätze**      | Kanonische Ressourcenreihenfolge, Zeitbegrenzung beim Sperren, asymmetrisches Verhalten, Butler (Semaphore). |

## Weitere Probleme in der nebenläufigen Programmierung

### Race Condition (Wettlaufsituation)

Ein Race Condition tritt auf, wenn das Ergebnis vom Timing oder der Reihenfolge abhängt, in der Threads auf gemeinsame Daten zugreifen.

#### Beispiel:

Zwei Threads erhöhen gleichzeitig denselben Zähler → Ergebnis kann falsch sein, wenn kein Lock verwendet wird.

#### Lösungsmöglichkeiten:

* **Locking** (Mutex, Semaphore)
* **Atomare Operationen**
* **Immutability** (unveränderliche Objekte)

### Livelock

Ein Livelock ähnelt einem Deadlock – nur dass die Threads nicht blockiert sind, sondern ständig in einer Schleife aktiv bleiben, ohne Fortschritt zu machen.

#### Beispiel:

Zwei Personen weichen sich gegenseitig ständig aus, um durch eine Tür zu gehen, blockieren sich dabei aber weiterhin.

#### Lösungsmöglichkeiten:

* Backoff-Strategien
* Zufallsverzögerungen

### Starvation (Verhungern)

Ein Thread bekommt nie CPU-Zeit oder keinen Zugriff auf Ressourcen, weil andere ständig bevorzugt werden.

#### Ursachen:

* Ungerechte Scheduler
* Prioritätsinversion (niedrigpriorisierter Thread hält Ressource, die ein höherpriorisierter braucht)

#### Lösung:

* **Faire Locks**
* **Priority Inheritance**

### Race Hazard / Data Hazard

Kommt aus der Hardware-Programmierung (z. B. Pipelines in CPUs), aber ähnlich wie Race Conditions in Software.

### Priority Inversion

Ein niedriger Thread hält eine Ressource, die ein höherpriorisierter Thread benötigt, aber wird nicht unterbrochen → hoher Thread blockiert.

#### Lösung:

* **Priority Inheritance Protocol**

### Zusammenfassung als Tabelle

| Phänomen           | Ursache                                                   | Symptom                          | Lösung                 |
| ------------------ | --------------------------------------------------------- | -------------------------------- | ---------------------- |
| Deadlock           | Zirkuläres Warten auf Locks                               | Alle Threads warten unbegrenzt   | Feste Lock-Reihenfolge |
| Race Condition     | Kein synchroner Zugriff auf gemeinsame Daten              | Zufällige/fehlerhafte Ergebnisse | Locking, Atomics       |
| Livelock           | Threads reagieren ständig, machen aber keinen Fortschritt | CPU-Auslastung, kein Fortschritt | Zufall, Backoff        |
| Starvation         | Ungerechte Ressourcenzuweisung                            | Einzelne Threads arbeiten nie    | Faire Scheduler        |
| Priority Inversion | Niedriger Thread blockiert höheren                        | Hänger trotz hoher Priorität     | Priority Inheritance   |

## Lösungsansätze in C#

In C# gibt es verschiedene **Locking-Mechanismen**, um nebenläufige Zugriffe zu koordinieren und Race Conditions oder Deadlocks zu vermeiden. Hier ist eine Übersicht:

### 1. `lock`-Statement (Monitor)

Der einfachste und gebräuchlichste Mechanismus für exklusive Sperren.

```csharp
private readonly object _lockObj = new();

public void KritischerAbschnitt()
{
    lock (_lockObj)
    {
        // Nur ein Thread gleichzeitig hier drin
    }
}
```

#### Intern: verwendet `Monitor.Enter()` und `Monitor.Exit()`

### 2. `Monitor`-Klasse

Mehr Kontrolle als `lock`, z. B. mit Timeout oder TryEnter:

```csharp
if (Monitor.TryEnter(_lockObj, TimeSpan.FromSeconds(2)))
{
    try
    {
        // geschützter Zugriff
    }
    finally
    {
        Monitor.Exit(_lockObj);
    }
}
```

### 3. `Mutex` (Systemweite Sperre)

Kann auch über Prozessgrenzen hinweg sperren.

```csharp
Mutex mutex = new Mutex(false, "Global\\MeinMutex");

if (mutex.WaitOne(TimeSpan.FromSeconds(1)))
{
    try
    {
        // Zugriff gesichert
    }
    finally
    {
        mutex.ReleaseMutex();
    }
}
```

#### Nachteile:

* Teurer als `lock`
* Für interprozessuale Koordination gedacht

### 4. `Semaphore` & `SemaphoreSlim`

Erlaubt eine bestimmte Anzahl paralleler Zugriffe.

```csharp
SemaphoreSlim sem = new SemaphoreSlim(3); // max 3 Threads

await sem.WaitAsync();
try
{
    // geschützter Bereich
}
finally
{
    sem.Release();
}
```

#### Unterschied:

* `SemaphoreSlim` ist nur innerhalb eines Prozesses und performanter
* `Semaphore` auch systemweit möglich

### 5. `ReaderWriterLockSlim`

Optimiert für Szenarien mit vielen Lesern und wenigen Schreibern.

```csharp
ReaderWriterLockSlim rwLock = new();

public void Lesen()
{
    rwLock.EnterReadLock();
    try
    {
        // lesen
    }
    finally
    {
        rwLock.ExitReadLock();
    }
}

public void Schreiben()
{
    rwLock.EnterWriteLock();
    try
    {
        // schreiben
    }
    finally
    {
        rwLock.ExitWriteLock();
    }
}
```

### 6. `SpinLock` (System.Threading)

Leichtgewichtige Sperre, die aktiv wartet (Spinning) – sinnvoll nur bei sehr kurzer Sperrdauer und hohem Durchsatzbedarf.

```csharp
SpinLock spinLock = new();
bool lockTaken = false;

spinLock.Enter(ref lockTaken);
try
{
    // kritischer Abschnitt
}
finally
{
    if (lockTaken)
        spinLock.Exit();
}
```

### 7. `Interlocked` (Atomare Operationen, kein echter Lock)

Nützlich für einfache atomare Zähler oder Flags:

```csharp
int counter = 0;
Interlocked.Increment(ref counter);
```

### Zusammenfassung: Wann was?

| Mechanismus            | Use Case                         | Besonderheiten                    |
| ---------------------- | -------------------------------- | --------------------------------- |
| `lock`                 | Standard-Locking                 | Einfach & sicher                  |
| `Monitor`              | Feiner kontrolliertes Locking    | TryEnter, Timeout                 |
| `Mutex`                | Prozessübergreifendes Locking    | Teuer, aber mächtig               |
| `Semaphore(Slim)`      | Begrenzte gleichzeitige Zugriffe | Gut für Throttling                |
| `ReaderWriterLockSlim` | Viele Leser, wenige Schreiber    | Gute Performance                  |
| `SpinLock`             | Sehr kurze Sperren               | Aktives Warten, keine Blockierung |
| `Interlocked`          | Atomare Operationen              | Kein echter Lock, sehr performant |

## Beispiel: Dining Philosophers mit `SemaphoreSlim`

Hier ist ein **vollständiges, aber einfach gehaltenes C#-Beispiel** für das **Dining Philosophers Problem** mit Vermeidung von **Deadlocks** durch einen sogenannten **"Butler"**, der maximal vier der fünf Philosophen gleichzeitig Essen serviert. So wird ein zyklisches Warten verhindert.

In [1]:
using System;
using System.Threading;
using System.Threading.Tasks;

const int AnzahlPhilosophen = 5;
static object[] gabeln = new object[AnzahlPhilosophen];
static SemaphoreSlim butler = new SemaphoreSlim(AnzahlPhilosophen - 1); // Max. 4 dürfen gleichzeitig essen

for (int i = 0; i < AnzahlPhilosophen; i++)
    gabeln[i] = new object();

Task[] philosophen = new Task[AnzahlPhilosophen];
for (int i = 0; i < AnzahlPhilosophen; i++)
{
    int id = i;
    philosophen[i] = Task.Run(() => PhilosophDenktUndIsst(id));
}

Task.WaitAll(philosophen);

static void PhilosophDenktUndIsst(int id)
{
    for (int runde = 0; runde < 3; runde++)
    {
        Console.WriteLine($"Philosoph {id} denkt.");
        Thread.Sleep(new Random().Next(100, 500));

        butler.Wait(); // Maximal 4 Philosophen gleichzeitig reinlassen

        object linkeGabel = gabeln[id];
        object rechteGabel = gabeln[(id + 1) % AnzahlPhilosophen];

        // Immer kleinere ID zuerst greifen, um Ordnung zu garantieren
        object ersteGabel = id < (id + 1) % AnzahlPhilosophen ? linkeGabel : rechteGabel;
        object zweiteGabel = id < (id + 1) % AnzahlPhilosophen ? rechteGabel : linkeGabel;

        lock (ersteGabel)
        {
            lock (zweiteGabel)
            {
                Console.WriteLine($"Philosoph {id} isst (Runde {runde + 1}).");
                Thread.Sleep(new Random().Next(200, 500));
            }
        }

        Console.WriteLine($"Philosoph {id} ist fertig mit Essen.");
        butler.Release(); // Platz am Tisch freigeben
    }
}

Philosoph 0 denkt.
Philosoph 1 denkt.
Philosoph 3 denkt.
Philosoph 2 denkt.
Philosoph 4 denkt.
Philosoph 2 isst (Runde 1).
Philosoph 0 isst (Runde 1).
Philosoph 0 ist fertig mit Essen.
Philosoph 0 denkt.
Philosoph 4 isst (Runde 1).
Philosoph 2 ist fertig mit Essen.
Philosoph 2 denkt.
Philosoph 1 isst (Runde 1).
Philosoph 1 ist fertig mit Essen.
Philosoph 1 denkt.
Philosoph 3 isst (Runde 1).
Philosoph 4 ist fertig mit Essen.
Philosoph 4 denkt.
Philosoph 0 isst (Runde 2).
Philosoph 0 ist fertig mit Essen.
Philosoph 0 denkt.
Philosoph 3 ist fertig mit Essen.
Philosoph 3 denkt.
Philosoph 2 isst (Runde 2).
Philosoph 4 isst (Runde 2).
Philosoph 4 ist fertig mit Essen.
Philosoph 4 denkt.
Philosoph 2 ist fertig mit Essen.
Philosoph 2 denkt.
Philosoph 1 isst (Runde 2).
Philosoph 3 isst (Runde 2).
Philosoph 1 ist fertig mit Essen.
Philosoph 1 denkt.
Philosoph 0 isst (Runde 3).
Philosoph 3 ist fertig mit Essen.
Philosoph 3 denkt.
Philosoph 2 isst (Runde 3).
Philosoph 0 ist fertig mit Essen.
Philo

### Erklärungen:

| Teil                                   | Erklärung                                                                                      |
| -------------------------------------- | ---------------------------------------------------------------------------------------------- |
| `SemaphoreSlim(AnzahlPhilosophen - 1)` | Verhindert, dass alle Philosophen gleichzeitig Gabeln wollen, wodurch Deadlocks möglich wären. |
| `lock(ersteGabel) lock(zweiteGabel)`   | Sichert Gabeln in einer festen Reihenfolge.                                                    |
| `Task.Run(...)`                        | Startet jeden Philosophen asynchron.                                                           |
| `Thread.Sleep(...)`                    | Simuliert Denken und Essen.                                                                    |
| `Console.WriteLine(...)`               | Zeigt den Status jedes Philosophen im Konsolenfenster.                                         |

### Deadlock-Vermeidung:

1. Der **Butler** (Semaphore) verhindert, dass alle Philosophen gleichzeitig essen.
2. Durch das **geordnet Sperren** der Gabeln (immer erst kleinere ID), wird **zirkuläres Warten** unterbunden.